In [ ]:
## Notebook to load keras model and train it on spectrogram data

In [1]:
import os
# load data/models/conv2D_model.keras
from tensorflow import keras
working_dir = os.getenv("WORKING_DIR", ".")
# model_path = os.path.join(working_dir, "data", "models", "conv2D_model.keras")
# model = keras.models.load_model(model_path)
# model.summary()

2025-12-11 12:09:30.332031: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-11 12:09:30.405526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-11 12:09:35.951702: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [22]:
import sys

WORKING_DIR = os.getenv("WORKING_DIR")
sys.path.append(WORKING_DIR)
from databass.models.conv2D import create_model
model = create_model((128, 128, 1))

/home/julien/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │         3,612 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,307,676 (12.62 MB)

 Trainable params: 3,307,676 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
path_data = os.path.join(working_dir, "data", "spectrograms")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the spectrograms from each subfolders
# Each subfolder is a note label
# In them there are .npy files with spectrogram data
# (e.g. data/spectrograms/A4/xxx.npy)
# x in subfolders name are # notes
X = [] = []
labels = []
for label in os.listdir(path_data):
    label_path = os.path.join(path_data, label)
    if os.path.isdir(label_path):
        print(f"Loading label: {label}")
        for file in os.listdir(label_path):
            if file.endswith(".npy"):
                file_path = os.path.join(label_path, file)
                spectrogram = np.load(file_path)
                X.append(spectrogram)
                labels.append(label)
X = np.array(X)
labels = np.array(labels)


Loading label: G3
Loading label: D2
Loading label: D3
Loading label: G1
Loading label: C#3
Loading label: B1
Loading label: E2
Loading label: C3
Loading label: B2
Loading label: F#3
Loading label: F2
Loading label: F1
Loading label: G#1
Loading label: E1
Loading label: A1
Loading label: G#2
Loading label: G2
Loading label: E3
Loading label: C2
Loading label: A#2
Loading label: A#1
Loading label: F#1
Loading label: C#2
Loading label: F3
Loading label: F#2
Loading label: A2
Loading label: D#2
Loading label: D#3


In [ ]:
# import pickle
# le_path = os.path.join(working_dir, 'data', 'models', 'conv2D_label_encoder.pkl')

# with open(le_path, 'rb') as f:
#     le = pickle.load(f)

27


(20592,)

In [16]:
le = LabelEncoder()
y = le.fit_transform(labels)


In [17]:
print(labels[0])
print(y[0])
y.shape


G3
27


(20592,)

In [18]:
print(X[0])
print(X.shape)

[[[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.01482477]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.00564823]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.02446556]]

 ...

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]]
(20592, 128, 128, 1)


In [20]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)


In [21]:
# Check X shape
print(f"X shape: {X_train.shape}")
print(f"y shape: {y_train.shape}")
y_train[0]

X shape: (16473, 128, 128, 1)
y shape: (16473,)


np.int64(10)

In [ ]:

# use validation_data instead of validation_split to avoid extra tf.function calls that can trigger
# "Creating variables on a non-first call to a function decorated with tf.function."
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
)

Epoch 1/10
 10/515 ━━━━━━━━━━━━━━━━━━━━ 8:26 1s/step - accuracy: 0.0296 - loss: 3.3320 

KeyboardInterrupt: 